# Implementacja szyfru (kod wraz z komentarzami) (podrozdzial roboczy)

Poniżej zostanie przedstawiona implementacja szyfru ADFGVX wraz z komentarzami. Jest on zaimplementowany w języku python 3 w trzech plikach. Pierwszy plik Encode.py przedstawia sposób szyfrowania, w pliku Decode.py przedstawiony jest sposób dekodowania a trzeci plik Driver.py zawiera main.

#### Encode.py ####

In [ ]:
# re - modul do pracy z wyrazeniami regularnymi
import re
# shuffle - wykorzystywane do mieszania elementow listy
from random import shuffle
# digits - cyfry od 0 do 9
# ascii_uppercase - wielkie litery alfabetu angielskiego 
from string import digits, ascii_uppercase


class Encode :
    # definiowanie konstruktora dla klasy
    def __init__(self):
        pass
    
    # wejście:
    # txt_file_name - podana przez uzytkownika sciezka do pliku w ktorym znajduja sie dane do szachownicy
    # wyjscie:
    # grid - uzupelniona szachownica
    # uzupełnienie szachownicy znakami 'A-Z' oraz cyframi '0-9'
    def genGrid(self):

        txt_file_name=input("\nPodaj ścieżkę pliku z danymi do uzupełnienia szachownicy Polibiusza: ")
        # otwarcie pliku do odczytu i zapis do zmeinnej txt_file
        txt_file=open(txt_file_name,"r")
        # odczytanie zawartosci pliku i zapis do zmiennej file_content
        file_content = txt_file.read()
        # podzial zawartosci pliku na ciagi znakow oddzielone przecinkami i zapis do zmiennej chars
        chars = file_content.split(",")
        # zamkniecie pliku 
        txt_file.close()
        print("\nWygenerowana szachownica Polibiusza:")
        # grid - zmienna globalna, lista skladajaca sie z 6 podlist w kazdej po 6 elementow
        global grid
        grid = [chars[0:6], chars[6:12], chars[12:18], chars[18:24], chars[24:30], chars[30:36]]
        # wypisanie na ekran uzupelnionej szachownicy
        print("    ", end = "")
        for char in ascii_uppercase[0:6]:
            print(char, end = "    ")
        print()
        for char, row in zip(ascii_uppercase[0:6], grid):
            print(char, row)  
            
    # znajdowanie danej w szachownicy, zwracanie jej wspolrzednych 
    # rowNum - numer wiersza
    # colNum - numer kolumny
    # value - wartosc dla ktorej wyznaczamy wspolrzedne
    def findVal(self, grid, value):
        # enumerate() - zwraca iterator:
        # do rowNum przypisywana jest wartosc wiersza, ktory jest aktualnie przetwarzany
        for rowNum, row in enumerate(grid):
            # do colNum przypisywana jest wartosc kolumny, w ktorej przetwarzany aktualnie element znjaduje się w wybranym wierszu
            for colNum, element in enumerate(row):
                if element == value:
                    return (rowNum, colNum)
                
    # konwersja podanej litery na odpowiadajaca jej litere
    def numToChar(self, number):
        if 0 <= number <= 5:
            return ascii_uppercase[number]
        else:
            raise ValueError('Number out of bounds')            
    
    # Tworzenie pierwszego szyfrogramu
    def genCipher(self, msg):
        self.genGrid()
        global firstCipher
        firstCipher = ""
        global originalMsg
        originalMsg = msg
        # usuwanie znaków, które nie są alfanumeryczne 
        originalMsg = re.sub(r"[^a-zA-Z0-9]","", originalMsg).upper()
        # przekształcanie na wielkie litery
        originalMsg = originalMsg.upper()
        
        for char in originalMsg:
            row, col = map(self.numToChar, self.findVal(grid, char))
            firstCipher = firstCipher + row + col
            
        return firstCipher 
    
    def genKey(self, inputKey):
        global firstCipher
        firstCipher = list(firstCipher)
        
        global cipherKey
        cipherKey = inputKey.upper()
        
        print("\nPOMOCNICZA TABLICA TRANSPOZYCJI")
        # keyWordGrid - dwuwymiarowa lista, ktora zostaje uzupelniona po wierszach fragmentami listy fistCipher
        # dlugosc wycinkow jest rowna dlugosci klucza szyfrowania; fragmenty te tworza kolumny tabeli szfrujacej
        # i sa indeksowane za pomoca liter klucza szyfrujacego
        keyWordGrid = [firstCipher[i:i+len(cipherKey)] for i in range(0,len(firstCipher),len(cipherKey))]
        print("  ", end = "")
        for char in cipherKey:
            print(char, end = "    ")
        print()
        for row in keyWordGrid:
            print(row)
        print()
        
        # posortowanie listy skladajacej sie z liter klucza szyfrujacego i uzyskanie cipherKeySorted
        indices = sorted(range(len(cipherKey)), key=lambda i: cipherKey[i])
        cipherKeySorted = ([cipherKey[i] for i in indices])
        
        print("POSORTOWANA TABLICA")
        # posortowanie keyWordGrid wedlug posortowanej listy cipherKeySorted i uzyskanie keyWordGridSorted
        keyWordGridSorted = ([[row[i] for i in indices] for row in keyWordGrid])
        # wypisanie posortowanej tablicy
        print("  ", end = "")
        for char in cipherKeySorted:
            print(char, end = "    ")
        print()
        for row in keyWordGridSorted:
            print(row)
        
        finalCipherTemp = [[row[col] for row in keyWordGridSorted] for col in range(len(row))]
    
        finalCipher = [item for sublist in finalCipherTemp for item in sublist]
        # laczenie elementow listy w jeden ciag znakow, ktory ostatecznie jest tekstem zaszyfrowanym
        finalCipher = "".join(finalCipher)
    
        return finalCipher
    
    def getGrid(self):
        # zwraca nam grid - szachownice
        return grid
    
    def getKey(self):
        # zwraca nam cipherKey - klucz szyfrujacy
        return cipherKey
    
    def getFirstCipher(self):
        #zwraca nam firstCipher - pierwsze zaszyfrowanie
        return firstCipher

#### Decode.py ####

In [ ]:
class Decode:
    # definiowanie konstruktora dla klasy
    def __init__(self):
        pass
        
    def decrypt(self, gridCopy, wordCopy, finalCipherCopy):
        grid = gridCopy
        cipherKey = wordCopy
        finalCipher = finalCipherCopy
        
        # sortowanie indeksow klucza szyfrowania w kolejnosci alfabetycznej 
        indices = sorted(range(len(cipherKey)), key=lambda i: cipherKey[i])
        
        gridColumnLen= len(finalCipher) // len(cipherKey)
        
        # tworzenie listy, dzielimy tekst na fragmenty kolumny o okreslonej dlugosci 
        gridColumns = [finalCipher[i:i+gridColumnLen] for i in range(0,len(finalCipher),gridColumnLen)]
        
        global keyWordGridSorted
        # keyWordGridSorted - tworzymy przez zamiane wierszy na kolumny
        keyWordGridSorted = list(zip(*gridColumns))
    
        print("POSORTOWANA TABLICA TRANSPOZYCJI")
        global cipherKeySorted
        # sortowanie listy zawierajacej klucz
        cipherKeySorted = ([cipherKey[i] for i in indices])
        # wypisanie posortowanej listy
        print("  ", end = "")
        for char in cipherKeySorted:
            print(char, end = "    ")
        print()
        
        for row in keyWordGridSorted:
            print(row)
        print()
        
        # odwrocenie sortowania kolumn siatki
        global indicesDecoded
        indicesDecoded = sorted(range(len(cipherKeySorted)), key=lambda i: indices[i])
        
        # wypisz oryginalna tablice transpozycji
        print("ORYGINALNA TABLICA TRANSPOZYCJI")
        global keyWordGrid
        # ustawianie kolumn w posortowanej tablicy transpozycji tak aby pirwszy wiersz tablicy byl zgodny z kluczem
        keyWordGrid = ([[row[i] for i in indicesDecoded] for row in keyWordGridSorted])
        print("  ", end = "")
        for char in cipherKey:
            print(char, end = "    ")
        print()
        for row in keyWordGrid:
            print(row)
        
        # wypisanie tekstu odczytaego z oryginalnej transpozycji
        global firstCipher
        # zwracanie po kolei elementow z wierszy
        firstCipher = [item for sublist in keyWordGrid for item in sublist]
        firstCipher = "".join(firstCipher)
     #   print("\nTEKST TYMCZASOWY: ",firstCipher)
        
        # tworzenie listy par bedacych wspolrzednymi znakow 
        firstCipherList = [firstCipher[i:i+2] for i in range(0, len(firstCipher), 2)]
                
        # zamiana pary znakow reprezentuacych pozycje w siatce Polibiusza na dany znak z tej siatki
        firstCipherListInt = [[ord((char.lower())) - 97 for char in element] for element in firstCipherList]
        
        global originalMsg
        originalMsg = ""

        for y, x in firstCipherListInt:
            originalMsg = originalMsg + grid[y][x]
        
        return originalMsg

#### Driver.py ####

In [ ]:
from Encode import Encode
from Decode import Decode

enc = Encode()
dec = Decode()

choice = ""
originalMsg = ""
firstCipher = ""
cipherKey = ""
cipher = ""
finalCipher = ""
decodedMsg = ""

while choice != "4":
    print("\nWybierz jedna z opcji: ")
    print("1. Podaj wiadomosc do zakodowania ")
    print("2. Podaj klucz do ostatecznego zaszyfrowania wiadomosci ")
    print("3. Podaj zaszyforwana wiadomosc do zdeszyfrowania ")
    print("4. Wyjscie ")
    
    choice = input("Wybór: ")
    
    if (choice == "1"):
        #while not 0 < len(originalMsg) <= 80:
        #    originalMsg = input("Wpisz wiadomosc do zakodowania ")
        msg_file_name = input("\nPodaj ścieżkę do pliku z wiadomością do zakodowania: ")
        # otwarcie do odczytu pliku z tekstem do zaszyfrowania 
        msg_file = open(msg_file_name,"r")
        # originMsg - wiadomosc z pliku do zaszyfrowania 
        originalMsg = msg_file.read()
        #print("\nzawartość pliku:\n",originalMsg)
        # firstCipher - tekst po pierszej czesci
        firstCipher = enc.genCipher(originalMsg)
        #print("\nPierwszy szyfrogram: " + firstCipher)
        
    elif (choice == "2"):
        print("\nDługość pierwszego szyfrogramu:", len(enc.getFirstCipher()))
        while len(cipherKey) == 0 or len(enc.getFirstCipher()) % len(cipherKey) != 0:
            cipherKey = input("\nPodaj klucz transpozycji (Dzielnik długości pierwszego szyfrogramu): ")
        # cipher - ostateczny zaszyfrowany tekst
        cipher = enc.genKey(cipherKey)
        ciph_file = open("szyfrogram.txt", "w")
        ciph_file.write(cipher)
        ciph_file.close()
        #print("\nOstateczny szyfrogram: " + cipher)
        print("\nSzyfrogram zapisany został w pliku szyfrogram.txt")

    elif (choice == "3"):
        # finalCipher - zaszyfrowany tekst to zdeszyforwania
        to_dec_file_name = input("\nPodaj ścieżkę do pliku z szyfrogramem: ")
        to_dec_file = open(to_dec_file_name,"r") 
        finalCipher = to_dec_file.read()
        # decodeMsg - zdeszyfrowany tekst
        decodedMsg = dec.decrypt(enc.getGrid(), enc.getKey(), finalCipher)
        #print("\nWiadomosc jawna: " + decodedMsg)
        dec_file = open("zdekodowany.txt", "w")
        dec_file.write(decodedMsg)
        dec_file.close()
        print("\nWiadomość jawna została zapisana w pliku zdekodowany.txt")
            
    elif (choice == "4"):
        quit()